# Radio Sources, Visibilities and Source Geometry

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import healpy as hp

In [2]:
sp.init_printing(use_latex='mathjax')
t = sp.Symbol('t', real=True)

## Source Geometry

Electric field sources are in angular co-ordinates ($\theta_0, \phi_0$), where $\theta$ is the azimuthal angle (measured from the zenith). Sources also have a well defined:

* frequency $\omega$, yes single frequencies here.
* phase offset, $\psi_0$
* amplitude, $E_o(t)$ (assume they're point sources for the moment).
* distance $R$ from the telescope.

In [3]:
theta0 = sp.Symbol('theta_0', real=True)
phi0 = sp.Symbol('phi_0', real=True)

# coordinates of the source...
R = sp.Symbol('R', real=True, positive=True)
pz = R*sp.cos(theta0)
px = R*sp.sin(theta0)*sp.sin(phi0)
py = R*sp.sin(theta0)*sp.cos(phi0)

# Distance to the source
d0 = sp.simplify(sp.sqrt(px**2 + py**2 + pz**2))   # Distance to source from the phase center
d0

R

We calculate the distance to the source from an antenna location $(x,y,z)$.

In [4]:
# Coordinates of the antenna
x = sp.Symbol('x', real=True)
y = sp.Symbol('y', real=True)
z = sp.Symbol('z', real=True) 
# Distance to source from position (x,y,z)
d1 = sp.expand(sp.sqrt((px-x)**2 + (py - y)**2 + (pz-z)**2)) 
d1

   ___________________________________________________________________________ ↪
  ╱  2    2        2        2    2        2        2    2                      ↪
╲╱  R ⋅sin (φ₀)⋅sin (θ₀) + R ⋅sin (θ₀)⋅cos (φ₀) + R ⋅cos (θ₀) - 2⋅R⋅x⋅sin(φ₀)⋅ ↪

↪ _______________________________________________________________
↪                                                    2    2    2 
↪ sin(θ₀) - 2⋅R⋅y⋅sin(θ₀)⋅cos(φ₀) - 2⋅R⋅z⋅cos(θ₀) + x  + y  + z  

In [5]:
print(sp.latex(d1))

\sqrt{R^{2} \sin^{2}{\left(\phi_{0} \right)} \sin^{2}{\left(\theta_{0} \right)} + R^{2} \sin^{2}{\left(\theta_{0} \right)} \cos^{2}{\left(\phi_{0} \right)} + R^{2} \cos^{2}{\left(\theta_{0} \right)} - 2 R x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} - 2 R y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} - 2 R z \cos{\left(\theta_{0} \right)} + x^{2} + y^{2} + z^{2}}


Calculate the path length difference between the source, $R$, and the antenna-source distance and take the limit as $R \rightarrow \infty$.

In [6]:
drR = sp.simplify(d0 -  d1)    # Change in distance at from antenna to source as a function of R
print(sp.latex(drR))
drR

R - \sqrt{R^{2} - 2 R x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} - 2 R y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} - 2 R z \cos{\left(\theta_{0} \right)} + x^{2} + y^{2} + z^{2}}


       _______________________________________________________________________ ↪
      ╱  2                                                                     ↪
R - ╲╱  R  - 2⋅R⋅x⋅sin(φ₀)⋅sin(θ₀) - 2⋅R⋅y⋅sin(θ₀)⋅cos(φ₀) - 2⋅R⋅z⋅cos(θ₀) + x ↪

↪ ____________
↪ 2    2    2 
↪   + y  + z  

In [7]:
dr = sp.limit(drR, R, sp.oo, "-")
dr

x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀)

So we could specify the path length in terms of the "direction cosines", 

$$l = \sin(\phi)\sin(\theta)$$
$$m = \cos(\phi)\sin(\theta)$$ 
$$n=\cos(\theta)$$

Which looks like $dr = l x + m y + n z$, but why replace two variables with three?

In [8]:
print(sp.latex(dr))
print(sp.python(dr))

x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + z \cos{\left(\theta_{0} \right)}
x = Symbol('x')
phi_0 = Symbol('phi_0')
theta_0 = Symbol('theta_0')
y = Symbol('y')
z = Symbol('z')
e = x*sin(phi_0)*sin(theta_0) + y*sin(theta_0)*cos(phi_0) + z*cos(theta_0)


### Radio Source

We create a radio source object that knows how to calculate the path-length difference, and its electric field strength at any position $(x,y,z)$.

A source has a complex amplitude $E(t)$ that encodes it's slowly varying amplitude and phase.

In [9]:
c = sp.Symbol('c', real=True, positive=True)
wavelength = sp.Symbol('lambda', real=True, positive=True)

class Source:
    def __init__(self, th, ph, amplitude):
        self.theta = th
        self.phi = ph
        self.amplitude = amplitude

    def field(self, omega, x, y, z):
        global t
        # Define phase as zero at the origin (0,0,0) in the antenna coordinates. This will return the field
        # at a position (x,y,z) with phase determined by path length
        phi_0 = self.phi
        theta_0 = self.theta
        dr = x*sp.sin(phi_0)*sp.sin(theta_0) + y*sp.sin(theta_0)*sp.cos(phi_0) + z*sp.cos(theta_0)
        k = 2*sp.pi / wavelength
        #wavelength =  (2 * sp.pi *c) / (omega)   # c = f lambda   = (omega / 2 pi) * lambda. lambda = 2*pi*c / omega
        phase = 2*sp.pi*dr/wavelength
        return self.amplitude*sp.exp(sp.I*(omega*t + k*dr))

In [10]:
class Sky:
    def __init__(self):
        self.sources = []

    def add(self, s):
        self.sources.append(s)

Lets create the sky and add a point source to it.

In [11]:
sky = Sky()

theta0 = sp.Symbol('theta_0', real=True)
phi0 = sp.Symbol('phi_0', real=True)
omega = sp.Symbol('omega', real=True)
amplitude0 = sp.Function('E_0')(omega, t)
sky.add(Source(theta0, phi0, amplitude0))


In [12]:
class Telescope:
    def __init__(self, n, max_wl=10):
        """ positions in wavelengths with maximum number of wavelengths"""
        pos = (np.round(np.random.uniform(0, max_wl, (n,2))*1000.0))/1000.0
        self.positions = pos

In [13]:
telescope = Telescope(10)
N_time_steps = 50
OMEGA_0 = 0.1
freq0 = 2*np.pi*OMEGA_0*1.3
period0 = 1.0/freq0
T_max = N_time_steps*period0

## Radio Receiver

What does a radio receiver do? Well it does some fancy electronics that grabs the signal from around a frequency $\omega$ and converts it to a signal at a much lower frequency. This magic is called down-conversion or _mixing_ and a bit of filtering...

A receiver is at a location $(x,y,z)$ and its signal is the sum of all the electric fields from all the sources in the sky. The good old principle of superposition from first-year physics?

In [14]:
class Receiver:
    def __init__(self, x, y, z, omega):
        self.x = x
        self.y = y
        self.z = z
        self.omega = omega

    def field(self, sky):
        # Electric fields are generated at the position on the x-axis, with frequency omega
        efield = 0.0
        for s in sky.sources:
            efield += s.field(self.omega, self.x, self.y, self.z)
        return efield

    ''' The data from the reciever is mixed down to a baseband frequency omega_0
    '''
    def baseband(self, sky, omega_0):
        global t
        efield = self.field(sky)
        # bb = sp.simplify((sp.exp(sp.I*(self.omega-omega_0)*t)*efield) / sp.exp(sp.I*2*self.omega*t))
        bb = efield*sp.exp(-sp.I*(self.omega*t))
        return sp.simplify(bb)

In [15]:
'''
   Now add a receiver at position 'x'
'''
x = sp.Symbol('x', real=True)
y = sp.Symbol('y', real=True)
z = sp.Symbol('z', real=True)
omega = sp.Symbol('omega', real=True, positive=True)

rx = Receiver(x, y, z, omega)

In [16]:
rx.field(sky)

            ⎛      2⋅π⋅(x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀))⎞
          ⅈ⋅⎜ω⋅t + ───────────────────────────────────────────────────────⎟
            ⎝                                 λ                           ⎠
E₀(ω, t)⋅ℯ                                                                 

In [17]:
print(sp.latex(rx.field(sky)))

E_{0}{\left(\omega,t \right)} e^{i \left(\omega t + \frac{2 \pi \left(x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + z \cos{\left(\theta_{0} \right)}\right)}{\lambda}\right)}


In [18]:
omega_0 = omega # sp.Symbol('omega_0', real=True, positive=True)

bb = rx.baseband(sky, omega_0)
bb

          2⋅ⅈ⋅π⋅(x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀))
          ─────────────────────────────────────────────────────────
                                      λ                            
E₀(ω, t)⋅ℯ                                                         

In [19]:
print(sp.latex(bb))
re_bb_model = sp.simplify(bb) # sp.simplify(sp.re(bb))
re_bb_model

E_{0}{\left(\omega,t \right)} e^{\frac{2 i \pi \left(x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + z \cos{\left(\theta_{0} \right)}\right)}{\lambda}}


          2⋅ⅈ⋅π⋅(x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀))
          ─────────────────────────────────────────────────────────
                                      λ                            
E₀(ω, t)⋅ℯ                                                         

## Radio Astronomy

Can we work out where a source is from this information? Not really. If we measure the signal, we'd have to know the exact form of $E_0(t)$ to work out it's coordinates, $(\theta_0, \phi_0)$. If we averaged $E(t)$ we'd get zero!

:(

theta_actual, phi_actual, phase_actual = sp.pi/6, 2*sp.pi/3, 0.45 #np.random.uniform(-np.pi/2., np.pi/2.),  np.random.uniform(0, np.pi*2.)
theta_actual2, phi_actual2, phase_actual2 = sp.pi/12, sp.pi/3, 1.45 #np.random.uniform(-np.pi/2., np.pi/2.),  np.random.uniform(0, np.pi*2.)
theta_deg = theta_actual*180.0 / sp.pi
phi_deg = phi_actual*180.0 / sp.pi

bb_actual1 = bb.subs([(theta0, theta_actual),(phi0, phi_actual), (phase0, phase_actual)])
bb_actual2 = bb.subs([(theta0, theta_actual2),(phi0, phi_actual2), (phase0, phase_actual2)])
bb_actual = bb_actual1 + bb_actual2

re_bb_data = sp.expand(bb_actual)
print('Real Baseband Data')
re_bb_data

## Visibility

To help with finding the sky from the measurements, we need something that doesn't average to zero.

The _visibility_ is the inner product of the signals from the two antennas. The antennas are located at $(x_1, y_1, z_1)$, and $(x_2, y_2, z_2)$.

$$ V_{1,2} = \int_t E_1(t) \bar{E_2}(t) dt $$

Where $\bar{E}(t)$ is the complex conjugate. 

#### Why the conjugate? 

Well imagine two fields with different phases $E_1 = e^{i (\omega t + \phi_1)}$, and $E_2 = e^{i (\omega t + \phi_2)} $. If we _didn't_ use the conjugate then the visibility would be:

$$ V = E_1 E_2 = e^{i (\omega t + \phi_1)} e^{i (\omega t + \phi_2)} = e^{i 2 \omega t}e^{i(\phi_1 + \phi_2)}$$

and the visibility would depend on the absolute phase - and the and wouldn't be very usedful as the choice of phase is arbitrary, and this expression would average to zero.

Using the conjugate makes it depend on the phase difference.

$$ V = E_1 \bar{E_2} = e^{i (\omega t + \phi_1)} e^{-i (\omega t + \phi_2)} = e^{i(\phi_1 - \phi_2)}$$

And this does not average to zero...

In [20]:
class Analytic_vis:
    def __init__(self, x_1, y_1, x_2, y_2, z_1, z_2, bb_model):
        self.bb1 = bb_model.subs([(x,x_1),(y, y_1),(z, z_1)])
        self.bb2 = bb_model.subs([(x,x_2),(y, y_2),(z, z_2)])
    
        self.vis = self.bb1 * sp.conjugate(self.bb2)

    def get_vis(self):
        return self.vis

In [21]:
x1 = sp.Symbol('x_1', real=True)
x2 = sp.Symbol('x_2', real=True)
y1 = sp.Symbol('y_1', real=True)
y2 = sp.Symbol('y_2', real=True)
z1 = sp.Symbol('z_1', real=True)
z2 = sp.Symbol('z_2', real=True)

analytic_vis = Analytic_vis(x1, y1, x2, y2, z1, z2, bb).get_vis()
sp.simplify(analytic_vis)

          2⋅ⅈ⋅π⋅(x₁⋅sin(φ₀)⋅sin(θ₀) - x₂⋅sin(φ₀)⋅sin(θ₀) + y₁⋅sin(θ₀)⋅cos(φ₀)  ↪
          ──────────────────────────────────────────────────────────────────── ↪
                                                                   λ           ↪
E₀(ω, t)⋅ℯ                                                                     ↪

↪ - y₂⋅sin(θ₀)⋅cos(φ₀) + z₁⋅cos(θ₀) - z₂⋅cos(θ₀))         
↪ ───────────────────────────────────────────────         
↪                                                 ________
↪                                                ⋅E₀(ω, t)

In [22]:
print(sp.latex(sp.simplify(analytic_vis)))

E_{0}{\left(\omega,t \right)} e^{\frac{2 i \pi \left(x_{1} \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} - x_{2} \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + y_{1} \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} - y_{2} \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + z_{1} \cos{\left(\theta_{0} \right)} - z_{2} \cos{\left(\theta_{0} \right)}\right)}{\lambda}} \overline{E_{0}{\left(\omega,t \right)}}


So the visibility for a source at a location is related to the magnitude of the electric field received at that location squared.

### Introducing the U-V-W plane

This visibility can be simplified by introducing quantities to represent the distance between the pair of antennas in the x,y and z directions, measured in wavelengths.

$u = \frac{(x_1 - x_2)}{\lambda}$, or $\lambda u = (x_1 - x_2)$

In [23]:
u = sp.Symbol('u')
v = sp.Symbol('v')
w = sp.Symbol('w')
avis_u = analytic_vis.subs([(x1,wavelength*u + x2),])
avis_v = avis_u.subs([(y1,wavelength*v + y2),])
avis_w = avis_v.subs([(z1,wavelength*w + z2),])
avis = sp.simplify(avis_w)
avis

          2⋅ⅈ⋅π⋅(u⋅sin(φ₀)⋅sin(θ₀) + v⋅sin(θ₀)⋅cos(φ₀) + w⋅cos(θ₀)) ________
E₀(ω, t)⋅ℯ                                                         ⋅E₀(ω, t)

In [24]:
print(sp.latex(avis))

E_{0}{\left(\omega,t \right)} e^{2 i \pi \left(u \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + v \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + w \cos{\left(\theta_{0} \right)}\right)} \overline{E_{0}{\left(\omega,t \right)}}


analytic_vis_actual = analytic_vis.subs([(theta0, theta_actual),(phi0, phi_actual)])
analytic_vis_actual

## Two Source Sky

Introduce a second source at position $\theta_1, \phi_1$ e.t.c.

In [25]:
theta1 = sp.Symbol('theta_1', real=True)
phi1 = sp.Symbol('phi_1', real=True)
amplitude1 = sp.Function('E_1')(omega, t)
sky.add(Source(theta1, phi1, amplitude1))

In [26]:
print(sp.latex(rx.baseband(sky, omega_0)))
rx.baseband(sky, omega_0)

\left(E_{0}{\left(\omega,t \right)} e^{\frac{i \left(\lambda \omega t + 2 \pi \left(x \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + y \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + z \cos{\left(\theta_{0} \right)}\right)\right)}{\lambda}} + E_{1}{\left(\omega,t \right)} e^{\frac{i \left(\lambda \omega t + 2 \pi \left(x \sin{\left(\phi_{1} \right)} \sin{\left(\theta_{1} \right)} + y \sin{\left(\theta_{1} \right)} \cos{\left(\phi_{1} \right)} + z \cos{\left(\theta_{1} \right)}\right)\right)}{\lambda}}\right) e^{- i \omega t}


⎛          ⅈ⋅(λ⋅ω⋅t + 2⋅π⋅(x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀))) ↪
⎜          ─────────────────────────────────────────────────────────────────── ↪
⎜                                           λ                                  ↪
⎝E₀(ω, t)⋅ℯ                                                                    ↪

↪              ⅈ⋅(λ⋅ω⋅t + 2⋅π⋅(x⋅sin(φ₁)⋅sin(θ₁) + y⋅sin(θ₁)⋅cos(φ₁) + z⋅cos(θ ↪
↪              ─────────────────────────────────────────────────────────────── ↪
↪                                               λ                              ↪
↪  + E₁(ω, t)⋅ℯ                                                                ↪

↪ ₁)))⎞        
↪ ────⎟        
↪     ⎟  -ⅈ⋅ω⋅t
↪     ⎠⋅ℯ      

In [27]:
bb = rx.baseband(sky, omega_0)
bb

⎛          ⅈ⋅(λ⋅ω⋅t + 2⋅π⋅(x⋅sin(φ₀)⋅sin(θ₀) + y⋅sin(θ₀)⋅cos(φ₀) + z⋅cos(θ₀))) ↪
⎜          ─────────────────────────────────────────────────────────────────── ↪
⎜                                           λ                                  ↪
⎝E₀(ω, t)⋅ℯ                                                                    ↪

↪              ⅈ⋅(λ⋅ω⋅t + 2⋅π⋅(x⋅sin(φ₁)⋅sin(θ₁) + y⋅sin(θ₁)⋅cos(φ₁) + z⋅cos(θ ↪
↪              ─────────────────────────────────────────────────────────────── ↪
↪                                               λ                              ↪
↪  + E₁(ω, t)⋅ℯ                                                                ↪

↪ ₁)))⎞        
↪ ────⎟        
↪     ⎟  -ⅈ⋅ω⋅t
↪     ⎠⋅ℯ      

In [28]:
analytic_vis = Analytic_vis(x1, y1, x2, y2, z1, z2, bb).get_vis()
analytic_vis = sp.expand(analytic_vis)
analytic_vis

          2⋅ⅈ⋅π⋅z₁⋅cos(θ₀)  -2⋅ⅈ⋅π⋅z₂⋅cos(θ₁)   2⋅ⅈ⋅π⋅x₁⋅sin(φ₀)⋅sin(θ₀)  -2⋅ⅈ ↪
          ────────────────  ──────────────────  ────────────────────────  ──── ↪
                 λ                  λ                      λ                   ↪
E₀(ω, t)⋅ℯ                ⋅ℯ                  ⋅ℯ                        ⋅ℯ     ↪

↪ ⋅π⋅x₂⋅sin(φ₁)⋅sin(θ₁)   2⋅ⅈ⋅π⋅y₁⋅sin(θ₀)⋅cos(φ₀)  -2⋅ⅈ⋅π⋅y₂⋅sin(θ₁)⋅cos(φ₁)  ↪
↪ ──────────────────────  ────────────────────────  ────────────────────────── ↪
↪         λ                          λ                          λ              ↪
↪                       ⋅ℯ                        ⋅ℯ                           ↪

↪                       2⋅ⅈ⋅π⋅z₁⋅cos(θ₀)  -2⋅ⅈ⋅π⋅z₂⋅cos(θ₀)   2⋅ⅈ⋅π⋅x₁⋅sin(φ₀) ↪
↪                       ────────────────  ──────────────────  ──────────────── ↪
↪  ________                    λ                  λ                      λ     ↪
↪ ⋅E₁(ω, t) + E₀(ω, t)⋅ℯ                ⋅ℯ                  ⋅ℯ                 ↪

↪ ⋅sin(θ₀)  -2⋅ⅈ⋅π⋅x₂⋅sin

In [29]:
avis_u = analytic_vis.subs([(x2,x1 - wavelength*u),])
avis_v = avis_u.subs([(y2,y1 - wavelength*v),])
avis_w = avis_v.subs([(z2,z1 - wavelength*w),])
avis = sp.simplify(avis_w)
#avis = avis_w
avis

                                                                               ↪
                                                                               ↪
          2⋅ⅈ⋅π⋅(u⋅sin(φ₀)⋅sin(θ₀) + v⋅sin(θ₀)⋅cos(φ₀) + w⋅cos(θ₀)) ________   ↪
E₀(ω, t)⋅ℯ                                                         ⋅E₀(ω, t) + ↪

↪            2⋅ⅈ⋅π⋅(x₁⋅sin(φ₀)⋅sin(θ₀) + y₁⋅sin(θ₀)⋅cos(φ₀) + z₁⋅cos(θ₀) + (λ⋅ ↪
↪            ───────────────────────────────────────────────────────────────── ↪
↪                                                                              ↪
↪  E₀(ω, t)⋅ℯ                                                                  ↪

↪ u - x₁)⋅sin(φ₁)⋅sin(θ₁) + (λ⋅v - y₁)⋅sin(θ₁)⋅cos(φ₁) + (λ⋅w - z₁)⋅cos(θ₁))   ↪
↪ ──────────────────────────────────────────────────────────────────────────   ↪
↪     λ                                                                      _ ↪
↪                                                                           ⋅E ↪

↪                        

Now we have to assume that the two sources are incoherent. In other words that the average of $\int E_0(t) E_1(t) \equiv 0$ which leaves the two terms independent.

The visibilities are therefore the sum of the visibiities from each independent source!

In [30]:
def term(expr, factor):
    return sp.simplify(factor*sp.expand(expr).coeff(factor))

vis = term(avis, amplitude0*sp.conjugate(amplitude0)) + term(avis, amplitude1*sp.conjugate(amplitude1))

vis

          2⋅ⅈ⋅π⋅(u⋅sin(φ₀)⋅sin(θ₀) + v⋅sin(θ₀)⋅cos(φ₀) + w⋅cos(θ₀)) ________   ↪
E₀(ω, t)⋅ℯ                                                         ⋅E₀(ω, t) + ↪

↪            2⋅ⅈ⋅π⋅(u⋅sin(φ₁)⋅sin(θ₁) + v⋅sin(θ₁)⋅cos(φ₁) + w⋅cos(θ₁)) _______ ↪
↪  E₁(ω, t)⋅ℯ                                                         ⋅E₁(ω, t ↪

↪ _
↪ )

In [31]:
print(sp.latex(vis))

E_{0}{\left(\omega,t \right)} e^{2 i \pi \left(u \sin{\left(\phi_{0} \right)} \sin{\left(\theta_{0} \right)} + v \sin{\left(\theta_{0} \right)} \cos{\left(\phi_{0} \right)} + w \cos{\left(\theta_{0} \right)}\right)} \overline{E_{0}{\left(\omega,t \right)}} + E_{1}{\left(\omega,t \right)} e^{2 i \pi \left(u \sin{\left(\phi_{1} \right)} \sin{\left(\theta_{1} \right)} + v \sin{\left(\theta_{1} \right)} \cos{\left(\phi_{1} \right)} + w \cos{\left(\theta_{1} \right)}\right)} \overline{E_{1}{\left(\omega,t \right)}}
